This notebook depends on another notebook that shows how we finetuned gemma model for function calling `function_calling_finetune_gemma.ipynb`. [notebook](https://github.com/VladimerKhasia/ML-in-Notebooks/blob/main/custom%20GenAI%20and%20tools/function_calling_finetune_gemma.ipynb)

Reference to used [training data](https://huggingface.co/datasets/NickyNicky/function-calling_chatml_gemma_v1).


In [1]:
## This notebook depend on another notebook where we finetuned gemma model for function calling
## and saved it on the mounted google drive at "./gdrive/MyDrive/fine-tuned-gemma/model"
from google.colab import drive

drive.mount('/content/gdrive')   # run it again if it fails first time - yes that actualy happens in the begginging.

Mounted at /content/gdrive


In [2]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
## pay attention to langchain and pydantic version compatibility: https://python.langchain.com/v0.1/docs/guides/development/pydantic_compatibility
!pip install -qU transformers langchain>=0.0.267 langchain_community bs4
!pip install -qU accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.0 MB/s eta 0:00:00


In [4]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    GenerationConfig,
    TextIteratorStreamer,
    StoppingCriteria,
    StoppingCriteriaList,
)

# Gemma model

In [5]:
model_id = "google/gemma-1.1-2b-it"
path_to_finetuned = "./gdrive/MyDrive/fine-tuned-gemma/model"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)
model = AutoModelForCausalLM.from_pretrained(path_to_finetuned,
                                             device_map="auto" if device!=torch.device('cpu') else None,
                                            #  torch_dtype=torch.bfloat16
                                            )

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

This is how to use langchain ChatModel. It is possible to do the same with just gemma and huggingface with internal custom memory state and [here is the example of that](https://github.com/VladimerKhasia/fastapi_X/blob/main/app/ai_model/chat_service.py).

In [7]:
from typing import Any, AsyncIterator, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain_core.messages.ai import AIMessage
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from langchain_core.runnables import run_in_executor
from transformers import pipeline
import re
import json
from typing import Any


class ListOfTokensStoppingCriteria(StoppingCriteria):
    """
    Class to define a stopping criterion based on a list of specific tokens.
    """
    def __init__(self, tokenizer, stop_tokens):
        self.tokenizer = tokenizer
        # Encode each stop token and store their IDs in a list
        self.stop_token_ids_list = [tokenizer.encode(stop_token, add_special_tokens=False) for stop_token in stop_tokens]

    def __call__(self, input_ids, scores, **kwargs):
        # Check if the last tokens generated match any of the stop token sequences
        for stop_token_ids in self.stop_token_ids_list:
            len_stop_tokens = len(stop_token_ids)
            if len(input_ids[0]) >= len_stop_tokens:
                if input_ids[0, -len_stop_tokens:].tolist() == stop_token_ids:
                    return True
        return False

# Define a list of stop tokens
stop_tokens = ["<end_of_turn>"]

# Initialize the stopping criteria with the tokenizer and the list of stop tokens
stopping_criteria = ListOfTokensStoppingCriteria(tokenizer, stop_tokens)

# Add the custom stopping criteria to a StoppingCriteriaList
stopping_criteria_list = StoppingCriteriaList([stopping_criteria])



class GemmaChatModel(BaseChatModel):
    """
    A custom chat model powered by Gemma from Hugging Face, designed to be informative, comprehensive, and engaging.
    See the custom model guide here: https://python.langchain.com/docs/modules/model_io/chat/custom_chat_model/
    """

    model_name: str = "gemma_chat_model"  # Replace with the actual Gemma model name
    task: str = "conversational"  # Task for the pipeline (conversational or summarization)
    #temperature = 0.0
    #n: int = 1500
    model : Any = None
    tokenizer : Any = None
    generation_config = GenerationConfig(
              max_new_tokens = 1500,
              temperature=0.20,
              # top_p=0.55,
              top_k=3, #50,
              repetition_penalty=1.,
              do_sample=True,)
    stopping_criteria_list = stopping_criteria_list

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """
        Args:
            messages: The list of prompt messages.
            stop: Optional list of stop tokens.
            run_manager: Optional callback manager.
            **kwargs: Additional keyword arguments.

        Returns:
            A ChatResult object containing the generated response.
        """

        prompt = messages[-1].content #[: self.n]
        input_ids = self.tokenizer.encode(prompt,
                          return_tensors="pt",
                          add_special_tokens=False).to(self.model.device) ##self.tokenizer(prompt, return_tensors="pt").to(device)
        outputs = self.model.generate(generation_config=self.generation_config,
                         input_ids=input_ids,
                         stopping_criteria=self.stopping_criteria_list,) #self.model.generate(**input_ids, max_new_tokens=self.n)  # , temperature=self.temperature
        text = self.tokenizer.decode(outputs[0], skip_special_tokens=False)  #self.tokenizer.decode(outputs[0])
        #text = " ".join(text.split("\n"))

        start_index, end_index = text.find(""), text.rfind("")
        response = text[start_index+len(""):end_index].strip()

        message = AIMessage(content=response, additional_kwargs={}, response_metadata={"time_in_seconds": 3})
        return ChatResult(generations=[ChatGeneration(message=message)])

    @property
    def _llm_type(self) -> str:
        """
        Returns the type of language model used: "gemma_chat_model".
        """
        return "gemma_chat_model"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """
        Returns a dictionary of identifying parameters for LangChain callbacks.
        """
        return {"model_name": self.model_name, "task": self.task}

llm = GemmaChatModel()
llm.model = model               # This is simple but not production level way of doing things. It's just for avoiding colab run out of memory on CPU
llm.tokenizer = tokenizer

# Conversational agent

In [8]:
from typing import Any, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

from langchain_core.tools import tool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.agents.format_scratchpad import format_to_openai_functions

In [9]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [10]:
!pip install -qU wikipedia

  Preparing metadata (setup.py) ... done


In [11]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [12]:
@tool
def something_else(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    #print(type(query))
    return query[::-1]

In [13]:
tools = [get_current_temperature, search_wikipedia]  #, something_else
tool_map = {tool.name: tool for tool in tools}

def tool_chain(dictionary, tool_map=tool_map):
  chosen_tool = tool_map[dictionary["name"]]
  return itemgetter("arguments") | chosen_tool

functions = [format_tool_to_openai_function(f) for f in tools]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [58]:
system_prompt = "You are a helpful assistant with access to the functions, which you use if required."
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
])


import re

def ResponseParser(output):
  # Regular expression to find the last pair of <function_call> and </function_call>
  pattern = r'<function_call>(.*?)</function_call>'

  # Using re.findall to find all matches
  #string = output.return_values['output'] # if additionally using OpenAIFunctionsAgentOutputParser() #output.__dict__  dict_keys(['return_values', 'log', 'type'])
  string = output.content # if not using OpenAIFunctionsAgentOutputParser()
  matches = re.findall(pattern, string, re.DOTALL)

  # Extracting the last match
  if matches:
      last_match = matches[-1]
      #return json.loads(last_match.strip())
      #output.additional_kwargs = {'function_call' : json.loads(last_match.strip())}
      dictionary = json.loads(last_match.strip())
      dictionary["arguments"] = json.dumps(dictionary["arguments"]) # It is done for openAI specs
      output.additional_kwargs = {'function_call' : dictionary}
      #output.content = ''
      return output

## chain = prompt | llm | ResponseParser | RunnablePassthrough.assign(output=tool_chain) ## or instead just do: | tool_chain
chain = prompt | llm | ResponseParser | OpenAIFunctionsAgentOutputParser() #| RunnablePassthrough.assign(output=tool_chain) ## or instead just do: | tool_chain


In [59]:
input = "what is the weather in sf?" #"what does wikipedia say about Feyerabend"

In [60]:
def templater(input: str, functions: list = functions):

  input_template = f"""<bos><start_of_turn>system
  You are a helpful assistant with access to the following functions.
  Use them if required:
  <tool>
  {functions}
  </tool>

  To use these functions respond with:
  <function_call> {{"name": "function_name", "arguments": {{"arg_1": "value_1", "arg_2": "value_2", ...}}}} </function_call>

  Contains properties essential for the model to respond according to the tasks:
  <observation> {{"arg_1": "value_1", "arg_2": "value_2", "arg_3": "value_3", ...}} </observation>

  Edge cases you must handle:
  - If there are no functions that match the user request, you will respond politely that you cannot help.
  <end_of_turn>
  <start_of_turn>user
  {input}<end_of_turn>
  <start_of_turn>function_call
  """
  return input_template

result = chain.invoke({"input": templater(input)})

In [61]:
# result.__dict__.keys() without OpenAIFunctionsAgentOutputParser:
# dict_keys(['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'example', 'tool_calls', 'invalid_tool_calls', 'usage_metadata'])
result.__dict__.keys()  # with OpenAIFunctionsAgentOutputParser:   dict_keys(['tool', 'tool_input', 'log', 'type', 'message_log'])
# result.message_log[0]  #dict_keys(['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'example', 'tool_calls', 'invalid_tool_calls', 'usage_metadata'])
result.message_log[0].content

'<bos><start_of_turn>system\n  You are a helpful assistant with access to the following functions.\n  Use them if required:\n  <tool>\n  [{\'name\': \'get_current_temperature\', \'description\': \'Fetch current temperature for given coordinates.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'latitude\': {\'description\': \'Latitude of the location to fetch weather data for\', \'type\': \'number\'}, \'longitude\': {\'description\': \'Longitude of the location to fetch weather data for\', \'type\': \'number\'}}, \'required\': [\'latitude\', \'longitude\']}}, {\'name\': \'search_wikipedia\', \'description\': \'Run Wikipedia search and get page summaries.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\'}}, \'required\': [\'query\']}}]\n  </tool>\n\n  To use these functions respond with:\n  <function_call> {"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": "value_2", ...}} </function_call>\n\n  Contains properties es

In [62]:
import re

latest_output = re.split(r'(<start_of_turn>user)', result.message_log[0].content)
latest_output[0]
if len(latest_output) > 2:
    header = latest_output[0]
    output = ''.join(latest_output[1:])
header

'<bos><start_of_turn>system\n  You are a helpful assistant with access to the following functions.\n  Use them if required:\n  <tool>\n  [{\'name\': \'get_current_temperature\', \'description\': \'Fetch current temperature for given coordinates.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'latitude\': {\'description\': \'Latitude of the location to fetch weather data for\', \'type\': \'number\'}, \'longitude\': {\'description\': \'Longitude of the location to fetch weather data for\', \'type\': \'number\'}}, \'required\': [\'latitude\', \'longitude\']}}, {\'name\': \'search_wikipedia\', \'description\': \'Run Wikipedia search and get page summaries.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\'}}, \'required\': [\'query\']}}]\n  </tool>\n\n  To use these functions respond with:\n  <function_call> {"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": "value_2", ...}} </function_call>\n\n  Contains properties es

In [63]:
output

'<start_of_turn>user\n  what is the weather in sf?<end_of_turn>\n  <start_of_turn>function_call\n  <function_call> {"name": "get_current_temperature", "arguments": {"latitude": 37.7833, "longitude": -122.4167}} </function_call><end_of_turn>'

In [64]:
#dict_keys(['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'example', 'tool_calls', 'invalid_tool_calls', 'usage_metadata'])
function_call = result.message_log[0].additional_kwargs
function_call

{'function_call': {'name': 'get_current_temperature',
  'arguments': '{"latitude": 37.7833, "longitude": -122.4167}'}}

In [65]:
result.message_log[0].model_output = "Hi"  # there is no model_output by default but we can create it
result.message_log[0].model_output

'Hi'

In [66]:
model_output = tool_map[result.tool].run(result.tool_input)
model_output

'The current temperature is 13.5°C'

In [67]:
class Memory(BaseModel):
  history: str = Field(default="")
  api_memory: List[dict] | list = Field(default=[])

memory = Memory()
memory.history

''

In [68]:
import re

def splitter(result):
  latest_output = re.split(r'(<start_of_turn>user)', result.message_log[0].content)
  if len(latest_output) > 2:
      header = latest_output[0]
      output = ''.join(latest_output[1:])
  return (header, output)

In [69]:
def agent(user_input, templater=templater, chain=chain, tool_map=tool_map,
          memory=memory, splitter=splitter, max_symbols=20000):

  #while True:
    input = memory.history + user_input
    result = chain.invoke({"input": templater(input)})
    model_output = tool_map[result.tool].run(result.tool_input)
    header, output = splitter(result)
    memory.history += output + " <start_of_turn>model\n  " + model_output + "<end_of_turn>\n"

    result.message_log[0].user_input = user_input
    result.message_log[0].model_output = model_output

    if len(memory.history) > max_symbols:
      print("Maximal memory storage used")
      memory.history = ''
      return #memory.history

    return result

In [70]:
def printer(result):
  return print(f"""user input:\n\n{result.message_log[0].user_input}, \n\nused tool:\n\n{result.tool},
          \n\nproposed data for the tool:\n\n{result.tool_input}, \n\nresult:\n\n {result.message_log[0].model_output}""")

In [71]:
input = "what is the weather in sf?" #"what does wikipedia say about Feyerabend"
result1 = agent(input)

printer(result1)

user input:

what is the weather in sf?, 

used tool:

get_current_temperature,
          

proposed data for the tool:

{'latitude': 37.78, 'longitude': -122.42}, 

result:

 The current temperature is 13.6°C


In [52]:
input = "what is the weather in Tbilisi?"
result2 = agent(input)

printer(result2)

user input:

what is the weather in Tbilisi?, 

used tool:

get_current_temperature,
          

proposed data for the tool:

{'latitude': 43.73, 'longitude': 43.65}, 

result:

 The current temperature is 34.2°C


In [72]:
result2.message_log[0].content

'<bos><start_of_turn>system\n  You are a helpful assistant with access to the following functions.\n  Use them if required:\n  <tool>\n  [{\'name\': \'get_current_temperature\', \'description\': \'Fetch current temperature for given coordinates.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'latitude\': {\'description\': \'Latitude of the location to fetch weather data for\', \'type\': \'number\'}, \'longitude\': {\'description\': \'Longitude of the location to fetch weather data for\', \'type\': \'number\'}}, \'required\': [\'latitude\', \'longitude\']}}, {\'name\': \'search_wikipedia\', \'description\': \'Run Wikipedia search and get page summaries.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\'}}, \'required\': [\'query\']}}]\n  </tool>\n\n  To use these functions respond with:\n  <function_call> {"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": "value_2", ...}} </function_call>\n\n  Contains properties es

'<bos><start_of_turn>system\n  You are a helpful assistant with access to the following functions.\n  Use them if required:\n  <tool>\n  [{\'name\': \'get_current_temperature\', \'description\': \'Fetch current temperature for given coordinates.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'latitude\': {\'description\': \'Latitude of the location to fetch weather data for\', \'type\': \'number\'}, \'longitude\': {\'description\': \'Longitude of the location to fetch weather data for\', \'type\': \'number\'}}, \'required\': [\'latitude\', \'longitude\']}}, {\'name\': \'search_wikipedia\', \'description\': \'Run Wikipedia search and get page summaries.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\'}}, \'required\': [\'query\']}}]\n  </tool>\n\n  To use these functions respond with:\n  <function_call> {"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": "value_2", ...}} </function_call>\n\n  Contains properties es

In [82]:
class ChatState:
  """
  """
  def __init__(self, tools=tools, chain=chain, memory = Memory(), max_symbols=20000):
    self.tools = tools
    self.functions = [format_tool_to_openai_function(f) for f in self.tools]
    self.tool_map = {tool.name: tool for tool in self.tools}
    self.chain = chain
    self.memory = memory
    self.max_symbols = max_symbols

  def templater(self, input: str):
    input_template = f"""<bos><start_of_turn>system
    You are a helpful assistant with access to the following functions.
    Use them if required:
    <tool>
    {self.functions}
    </tool>

    To use these functions respond with:
    <function_call> {{"name": "function_name", "arguments": {{"arg_1": "value_1", "arg_2": "value_2", ...}}}} </function_call>

    Contains properties essential for the model to respond according to the tasks:
    <observation> {{"arg_1": "value_1", "arg_2": "value_2", "arg_3": "value_3", ...}} </observation>

    Edge cases you must handle:
    - If there are no functions that match the user request, you will respond politely that you cannot help.
    <end_of_turn>
    <start_of_turn>user
    {input}<end_of_turn>
    <start_of_turn>function_call
    """
    return input_template

  def splitter(self):
    latest_output = re.split(r'(<start_of_turn>user)', self.result.message_log[0].content)
    if len(latest_output) > 2:
        header = latest_output[0]
        output = ''.join(latest_output[1:])
    return (header, output)


  def agent(self, user_input):

      input = self.memory.history + user_input
      self.result = self.chain.invoke({"input": self.templater(input)})
      model_output = self.tool_map[self.result.tool].run(self.result.tool_input)
      header, output = self.splitter()
      self.memory.history += output + " <start_of_turn>model\n  " + model_output + "<end_of_turn>\n"

      self.memory.api_memory.append({"user": user_input,
                                     "model": model_output,
                                     "tool": self.result.tool,
                                     "tool_input": self.result.tool_input
                                     })

      if len(self.memory.history) > self.max_symbols:
        #print("Maximal memory storage used")
        self.memory.history = ''
        del self.result
        return self.memory.api_memory

      return self.memory.api_memory


In [83]:
chatstate = ChatState()

input = "what is the weather in Tbilisi?"
chatstate.agent(input)

[{'user': 'what is the weather in Tbilisi?',
  'model': 'The current temperature is 36.3°C',
  'tool': 'get_current_temperature',
  'tool_input': {'latitude': 43.75, 'longitude': 44.45}}]

In [84]:
chatstate.agent("what does wikipedia say about Feyerabend")

[{'user': 'what is the weather in Tbilisi?',
  'model': 'The current temperature is 36.3°C',
  'tool': 'get_current_temperature',
  'tool_input': {'latitude': 43.75, 'longitude': 44.45}},
 {'user': 'what does wikipedia say about Feyerabend',
  'model': 'Page: Paul Feyerabend\nSummary: Paul Karl Feyerabend (German: [ˈfaɪɐˌʔaːbm̩t]; January 13, 1924 – February 11, 1994) was an Austrian philosopher best known for his work in the philosophy of science. He started his academic career as lecturer in the philosophy of science at the University of Bristol (1955–1958); afterwards, he moved to the University of California, Berkeley, where he taught for three decades (1958–1989). At various points in his life, he held joint appointments at the University College London (1967–1970), the London School of Economics (1967), the FU Berlin (1968), Yale University (1969), the University of Auckland (1972, 1975), the University of Sussex (1974), and, finally, the ETH Zurich (1980–1990). He gave lectures 

LangChain team introduced functional conversation tutorial with OpenAI at deeplearning.ai [see here](https://www.deeplearning.ai/short-courses/functions-tools-agents-langchain/). 

That same tutorial with useful explanations can be found [here](https://teetracker.medium.com/building-an-agent-from-scratch-with-langchain-2e1d1ef2f57f)
